# Imports

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import itertools
import numpy.matlib
import time

# Data Generation

In [2]:
n = 100000 # Number of samples
m = 1000 # Number of samples for validation/testing dataset
d = 8 # Dimension of feature variable vector
sigma = 0.01

np.random.seed(1) # Set seed for reproducibility
X = np.random.uniform(-1,1,(n + 2*m,d)) # 8 dimensional random vector X uniformly distributed on [-1,1]
eps = np.random.normal(0,sigma,n + 2*m) # Gaussian noise

Y = 8 + X[:,0]**2 + X[:,1]*X[:,2] + np.cos(X[:,3]) + np.exp(X[:,4]*X[:,5]) + 0.1*X[:,6] + eps # Output computation
Y = Y.reshape((-1,1)) # Transpose output vector

data = np.concatenate((X,Y), axis=1) # First 8 columns as input and last column as output
data_df = pd.DataFrame(data) # Create dataframe of numpy matrix
data_df.columns = ['X_1','X_2','X_3','X_4','X_5','X_6','X_7','X_8','Y']
data_df

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,Y
0,-0.165956,0.440649,-0.999771,-0.395335,-0.706488,-0.815323,-0.627480,-0.308879,10.218395
1,-0.206465,0.077633,-0.161611,0.370439,-0.591096,0.756235,-0.945225,0.340935,9.505227
2,-0.165390,0.117380,-0.719226,-0.603797,0.601489,0.936523,-0.373152,0.384645,10.492027
3,0.752778,0.789213,-0.829912,-0.921890,-0.660339,0.756285,-0.803306,-0.157785,9.026440
4,0.915779,0.066331,0.383754,-0.368969,0.373002,0.669251,-0.963423,0.500289,10.966825
...,...,...,...,...,...,...,...,...,...
101995,-0.965163,-0.868066,0.687381,0.737635,0.201860,0.130245,-0.953401,0.992092,10.001159
101996,0.801107,0.875722,-0.990787,0.911076,0.353634,-0.470982,-0.985444,-0.509583,9.140565
101997,0.126903,-0.399359,-0.606135,-0.742466,-0.001000,-0.924128,-0.592043,0.104867,9.935730
101998,-0.102098,-0.232576,-0.926793,-0.189997,-0.359941,-0.060435,-0.958450,0.175319,10.154265


In [3]:
# Training data

x_train = pd.DataFrame(data[:n,0:d]).to_numpy()
y_train = pd.DataFrame(data[:n,d]).to_numpy()

# Validation data

x_val = pd.DataFrame(data[n:n + m,0:d]).to_numpy()
y_val = pd.DataFrame(data[n:n + m,d]).to_numpy()

# Testing data

x_test = pd.DataFrame(data[n + m:n + 2*m,0:d]).to_numpy()
y_test = pd.DataFrame(data[n + m:n + 2*m,d]).to_numpy()

# Training the Neural Network

In [6]:
tf.random.set_seed(1) # Set seed for reproducibility

# Neural network model

neurons = 25 # 25 neurons in the hidden layer

NN_model = tf.keras.models.Sequential()
NN_model.add(tf.keras.layers.Dense(neurons, activation=tf.nn.sigmoid, input_dim = d)) 
NN_model.add(tf.keras.layers.Dense(1))

NN_model.compile(optimizer='Adam', loss='mse')

# Training process

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=10**(-5), patience=5) # Early stopping criterion

NN_model.fit(x_train, y_train, batch_size=32, epochs=15, validation_data=(x_val, y_val), callbacks=[callback], verbose=1) # Training process

Epoch 1/150
3125/3125 [==============================] - 2s 544us/step - loss: 7.6113 - val_loss: 0.3413
Epoch 2/150
3125/3125 [==============================] - 2s 532us/step - loss: 0.3051 - val_loss: 0.2597
Epoch 3/150
3125/3125 [==============================] - 2s 528us/step - loss: 0.2271 - val_loss: 0.1951
Epoch 4/150
3125/3125 [==============================] - 2s 529us/step - loss: 0.1878 - val_loss: 0.1573
Epoch 5/150
3125/3125 [==============================] - 2s 530us/step - loss: 0.1115 - val_loss: 0.0622
Epoch 6/150
3125/3125 [==============================] - 2s 527us/step - loss: 0.0489 - val_loss: 0.0391
Epoch 7/150
3125/3125 [==============================] - 2s 527us/step - loss: 0.0284 - val_loss: 0.0191
Epoch 8/150
3125/3125 [==============================] - 2s 529us/step - loss: 0.0174 - val_loss: 0.0139
Epoch 9/150
3125/3125 [==============================] - 2s 531us/step - loss: 0.0131 - val_loss: 0.0120
Epoch 10/150
3125/3125 [==============================]